In [1]:
import pandas
from tqdm import tqdm

In [2]:
def getSpectralRecall(row):
    return(min(
        (
            100.0 * (
                row.loc['backbone_cleavage_score'] /
                (len(row.loc['sequence']) - 1))
        ),
        100.0
    ))
def getFDR(row):
    if row.loc['deltaForwardReverseScore'] < 0:
        return(0)
    return(1)
def getCond(row):
    if row.loc['category'] != 'nuORF':
        return('Canonical')
    if 'canonical' in row.loc['header']:
        return('Canonical')
    return('nuORF')

In [3]:
THRESHOLDS = {
    'DBC.NULL': [0, 0, 0, 0],
    'DBC.B721': [9, 29, 4, 44],
    'DBC.PS': [8, 29, 5, 44],
    'DBC.RPF': [6, 40, 5, 44],
    'DBC.RNA': [3, 29, 5, 44]
}
def fdrFilter(row, name):
    threshold = THRESHOLDS[name]
    if row.loc['condType'] == 'Canonical':
        return(True)
    if not row.loc['score'] > threshold[0]:
        return(False)
    if not row.loc['percent_scored_peak_intensity'] > threshold[1]:
        return(False)
    if not row.loc['backbone_cleavage_score'] > threshold[2]:
        return(False)
    if not row.loc['SpectralRecall'] > threshold[3]:
        return(False)
    return(True)

In [4]:
def updateTypes(pep, out, name):
    df = pandas.read_csv(
        pep,
        sep='\t',
        header=0,
        index_col=['Peptide:UID']
    )
    df['SpectralRecall'] = df.apply(getSpectralRecall, axis=1)
    df['condType'] = df.apply(getCond, axis=1)
    df['fdr'] = df.apply(getFDR, axis=1)
    df['fdrFilter'] = df.apply(fdrFilter, axis=1, args=(name, ))
    df.to_csv(
        out,
        sep='\t',
        header=True,
        index=True
    )

In [5]:
conditions = [
    'DBC.RNA',
    'DBC.RPF',
    'DBC.B721',
    'DBC.PS',
    'DBC.NULL'
]
for cond in tqdm(conditions):
    pep = '../../data/pep/' + cond + '.pep'
    out = '../../data/final/' + cond + '.pep'
    updateTypes(pep, out, cond)

100%|██████████| 5/5 [00:29<00:00,  5.93s/it]
